# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np
from abstract_matrix import AbstractMatrix

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4400586832
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

## Abstract Matrix of "Zeros"

In [3]:
mat1 = AbstractMatrix.zeros((2, 3), ps3)
mat1

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']]

## Random Abstract Matrix

In [4]:
AbstractMatrix.random((3, 3), ps3)

PSRing3 Matrix:
[['{0, 1}' '{1}' '{}']
 ['{0}' '{0, 1, 2}' '{1, 2}']
 ['{0, 1}' '{0}' '{0, 1}']]

## Test Matrix

In [5]:
mat2_array = [['{0, 1}', '{0, 2}', '{1, 2}'],
              ['{0}'   , '{1, 2}',    '{2}'],
              ['{0, 1}',     '{}',     '{}']]

In [6]:
mat2 = AbstractMatrix(mat2_array, ps3)
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

## Get/Set Matrix Element Values

In [7]:
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

In [8]:
mat2[0, 1]

'{0, 2}'

In [9]:
mat2[2, 2] = '{0, 1, 2}'
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{0, 1, 2}']]

## Matrix Minor

In [10]:
mnr2 = mat2.minor(0,0)
mnr2

PSRing3 Matrix:
[['{1, 2}' '{2}']
 ['{}' '{0, 1, 2}']]

## Abstract Cofactor Matrix

In [12]:
cof2 = mat2.cofactor_matrix()
cof2

PSRing3 Matrix:
[['{1, 2}' '{0}' '{1}']
 ['{0, 2}' '{0}' '{0}']
 ['{1}' '{}' '{0, 1}']]

## Abstract Matrix Transpose

In [13]:
cof2_trans = cof2.transpose()
cof2_trans

PSRing3 Matrix:
[['{1, 2}' '{0, 2}' '{1}']
 ['{0}' '{0}' '{}']
 ['{1}' '{0}' '{0, 1}']]

## Abstract Matrix Determinant

In [15]:
mat2.determinant()

'{1}'

Here's a breaksdown of the basic computations required to get the determinant.

NOTE: Addition & multiplication for the ring, ps3, is commutative, so the order of adds and mults is irrelevant, below.

In [16]:
minor_det_0 = ps3.sub(ps3.mult('{1, 2}', '{}'), ps3.mult('{2}', '{}'))
minor_det_0

'{}'

In [17]:
minor_det_1 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{2}', '{0, 1}'))
minor_det_1

'{}'

In [18]:
minor_det_2 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{1, 2}', '{0, 1}'))
minor_det_2

'{1}'

In [19]:
det = ps3.sub(ps3.add(ps3.mult('{0, 1}', minor_det_0),
                      ps3.mult('{1, 2}', minor_det_2)),
              ps3.mult('{0, 2}', minor_det_1))
det

'{1}'

In [20]:
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

In [22]:
mat2.minor(1, 1)

PSRing3 Matrix:
[['{0, 1}' '{1, 2}']
 ['{0, 1}' '{}']]

In [23]:
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

## Abstract Matrix Inverse

In [26]:
mat2_inv = mat2.matrix_inverse()
mat2_inv

PSRing3 Matrix:
[['{}' '{}' '{1}']
 ['{}' '{1}' '{}']
 ['{1}' '{}' '{1}']]

In [27]:
mat2 * mat2_inv

PSRing3 Matrix:
[['{1}' '{}' '{}']
 ['{}' '{1}' '{}']
 ['{}' '{}' '{1}']]

In [28]:
mat2_inv * mat2

PSRing3 Matrix:
[['{1}' '{}' '{}']
 ['{}' '{1}' '{}']
 ['{}' '{}' '{1}']]

## Matrix Multiplication

In [31]:
mat2 * mat2

PSRing3 Matrix:
[['{}' '{0, 2}' '{1, 2}']
 ['{0}' '{0, 1, 2}' '{2}']
 ['{0, 1}' '{0}' '{1}']]

In [32]:
try:
    mat2x2 = mat2 * mat2
    print(mat2x2)
except Exception as exc:
    print(exc)

[['{}' '{0, 2}' '{1, 2}']
 ['{0}' '{0, 1, 2}' '{2}']
 ['{0, 1}' '{0}' '{1}']]


In [33]:
mat2p2 = mat2 + mat2

mat2p2

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']
 ['{}' '{}' '{}']]

In [34]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

name 'mat3' is not defined


In [ ]:
mat2m2 = mat2 - mat2
mat2

### 2x2 Test

In [ ]:
mat2x3

In [ ]:
array_determinant(mat2x3.array, mat2x3.ring)

In [ ]:
mat2x3.determinant()

In [ ]:
ps3.mult('{0, 1, 2}', '{0, 1, 2}')

In [ ]:
ps3.mult('{1}', '{1}')

In [ ]:
ps3.sub('{0, 1, 2}', '{1}')

In [ ]:
array_cofactor(mat2x3.array, mat2x3.ring)

### 3x3 Test

In [ ]:
# rnd1 = AbstractMatrix.random((3,3),ps3)
# rnd1.array

In [ ]:
rnd1 = np.array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
                 ['{0, 2}', '{}', '{1}'],
                 ['{0}', '{1}', '{0, 1}']], dtype='<U32')

In [ ]:
array_determinant(rnd1, ps3)

In [ ]:
rnd1_mat = AbstractMatrix(rnd1, ps3)
rnd1_mat

In [ ]:
rnd1_mat.determinant()

In [ ]:
array_cofactor(rnd1, ps3)

In [ ]:
rnd1_mat_cof = rnd1_mat.cofactor_matrix()
rnd1_mat_cof

### 4x4 Test

In [ ]:
# rnd2 = AbstractMatrix.random((4,4),ps3)
# rnd2.array

In [ ]:
rnd2 = np.array([['{}', '{0, 1, 2}', '{0, 1, 2}', '{}'],
                 ['{0, 1}', '{0, 2}', '{1, 2}', '{2}'],
                 ['{0, 2}', '{}', '{}', '{0, 1}'],
                 ['{1}', '{0}', '{0, 2}', '{}']], dtype='<U32')

In [ ]:
array_determinant(rnd2, ps3)

In [ ]:
array_cofactor(rnd2, ps3)

In [ ]:
rnd2_mat = AbstractMatrix(rnd2, ps3)

In [ ]:
rnd2_mat.determinant()

In [ ]:
rnd2_mat.cofactor_matrix()

## Abstract Matrix Inverse

In [ ]:
rnd1_mat

In [ ]:
ps3.inv(rnd1_mat.determinant())

In [ ]:
rnd1_cof = rnd1_mat.cofactor_matrix()
rnd1_cof

In [ ]:
rnd1_cof_tr = rnd1_cof.transpose()
rnd1_cof_tr

In [ ]:
rnd1_cof_tr.scalar_mult(ps3.inv(rnd1_mat.determinant()))

In [ ]:
rnd1_mat_inv = rnd1_mat.matrix_inverse()
rnd1_mat_inv

In [ ]:
rnd1_mat * rnd1_mat_inv

In [ ]:
ex = alg.Examples(alg_dir)

In [ ]:
f4 = ex[9]
f4.about(use_table_names=True)

In [ ]:
# foo3 = AbstractMatrix.random((3, 3), f4)
# foo3

In [ ]:
foo3_arr = np.array([['0', '1+a', '1+a'],
                     ['a', '0', 'a'],
                     ['1+a', '1', '1']])
foo3 = AbstractMatrix(foo3_arr, f4)
foo3

In [ ]:
foo3.determinant()

In [ ]:
foo3_inv = foo3.matrix_inverse()
foo3_inv

In [ ]:
foo3 * foo3_inv

In [ ]:
foo3.algebra

In [ ]:
foo3_inv.algebra

In [ ]:
foo3.algebra == foo3_inv.algebra

In [ ]:
ps3.mult_table